In [43]:
import pandas as pd

In [44]:
df_chistes_humor = pd.read_csv("data/classification/chistes_y_memes_clasificados_completos.csv")

In [45]:
df_chistes_humor

,id_chiste,Chistes,nivel_risa
0,1,1.png,4
1,2,2.png,2
2,3,3.png,3
3,4,4.jpeg,3
4,5,5.jpeg,2
...,...,...,...
10099,10113,¿Por qué los franceses comen caracoles?\n Por...,3
10100,10114,¿Qué le dice el 1 al 10?\n Para ser como yo d...,1
10101,10115,¿Sabes qué le dice una piedra a otra?\n ¡La v...,2
10102,10116,¿Por qué está tan feliz siempre la escoba?\n ...,2


In [46]:
df_chistes_gpt1 = pd.read_csv("data/classification/Chistes_GPT.csv", encoding="latin1",sep=";")


In [47]:
df_chistes_gpt1

,ID,Descripcion
0,1,"Un nio toca concentrado su flauta en el auto,..."
1,2,"Un perro pequeo, con una peluca rubia mal pue..."
2,3,Una estatua con una expresin seria y la mano ...
3,4,Una cucaracha animada se sienta en un restaura...
4,5,Un perro pequeo con expresin de incomodidad ...
...,...,...
994,995,Un grupo de jugadoras de ftbol con expresin ...
995,996,"Una persona con sobrepeso bailando, acompaada..."
996,997,Un beb recin nacido llorando en una sala de ...
997,998,"Dos imgenes de personajes de anime comparan ""..."


In [48]:
def clean_text(text):
    # Decodificar bytes mal interpretados en texto
    text = text.replace('\x97', 'o').replace('\x96', 'ni').replace('\x87', 'a').replace('\x8e', 'e').replace('\x92', 'i').replace('\x81', 'u')
    return text

In [49]:
df_chistes_gpt1['Descripcion'] = df_chistes_gpt1['Descripcion'].apply(lambda x: clean_text(x))

In [50]:
df_chistes_gpt1.rename(columns={"Descripcion":"Chistes"},inplace=True)

In [51]:
df_chistes_gpt1.columns

Index(['ID', 'Chistes'], dtype='object')

In [52]:
df_chistes_gpt2 = pd.read_excel("data/classification/memesdescripcion.xlsx")


In [53]:
df_chistes_gpt2

,Chistes
0,Todo el mundo: se fue la luz.Colombia: mira a ...
1,"Feliz navidad, dentro de pocos dias me quedo s..."
2,Mi esposo dandome terapia de que tenemos que a...
3,Aprenderprogramacion estudiando una carrera: N...
4,Cliente despues de comprar el asiento al lado ...
...,...
1001,"PETER, dime algo bonito que me haga ver las es..."
1002,tengo miedo de salir a la calle y que el googl...
1003,Yo: que pasó con el codigo que estaba programa...
1004,esta muy claro que la maldad no tiene limite: ...


In [54]:
# Agregar una columna 'ID' que comience desde 1000
df_chistes_gpt2['ID'] = range(1000, 1000 + len(df_chistes_gpt2))

# Opcional: Verificar la columna ID
print(df_chistes_gpt2.head())


                                             Chistes    ID
0  Todo el mundo: se fue la luz.Colombia: mira a ...  1000
1  Feliz navidad, dentro de pocos dias me quedo s...  1001
2  Mi esposo dandome terapia de que tenemos que a...  1002
3  Aprenderprogramacion estudiando una carrera: N...  1003
4  Cliente despues de comprar el asiento al lado ...  1004


In [55]:
df_text_imagenes = pd.concat([df_chistes_gpt1,df_chistes_gpt2],ignore_index=True)

In [58]:
df_chistes_humor['ID'] = range(1, 1 + len(df_chistes_humor))

In [60]:
# Realizar la unión y reemplazo de valores en la columna 'Chistes'
df_result = (
    df_chistes_humor
    .merge(df_text_imagenes[['ID', 'Chistes']], on='ID', how='left', suffixes=('', '_imagen'))
)

# Reemplazar los valores de 'Chistes' con los de 'Chistes_imagen' si no son nulos
df_result['Chistes'] = df_result['Chistes_imagen'].combine_first(df_result['Chistes'])

# Eliminar la columna auxiliar 'Chistes_imagen'
df_result.drop(columns=['Chistes_imagen'], inplace=True)

df_result

,id_chiste,Chistes,nivel_risa,ID
0,1,Un ninio toca concentrado su flauta en el auto...,4,1
1,2,"Un perro pequenio, con una peluca rubia mal pu...",2,2
2,3,Una estatua con una expresion seria y la mano ...,3,3
3,4,Una cucaracha animada se sienta en un restaura...,3,4
4,5,Un perro pequenio con expresion de incomodidad...,2,5
...,...,...,...,...
10099,10113,¿Por qué los franceses comen caracoles?\n Por...,3,10100
10100,10114,¿Qué le dice el 1 al 10?\n Para ser como yo d...,1,10101
10101,10115,¿Sabes qué le dice una piedra a otra?\n ¡La v...,2,10102
10102,10116,¿Por qué está tan feliz siempre la escoba?\n ...,2,10103


In [66]:
df_result = df_result[~df_result["Chistes"].str.contains(r"^\d+\.(png|jpg|jpeg)$", na=False)]


C:\Users\acost\AppData\Local\Temp\ipykernel_47452\2225231087.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_result = df_result[~df_result["Chistes"].str.contains(r"^\d+\.(png|jpg|jpeg)$", na=False)]


In [68]:
df_result.reset_index(inplace=True)

In [71]:
df_result.drop(columns=['index'],inplace=True)

C:\Users\acost\AppData\Local\Temp\ipykernel_47452\33022820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result.drop(columns=['index'],inplace=True)


In [63]:
df_result.drop(columns=["ID"],inplace=True)

In [72]:
df_result.to_csv("complete_dataset_chistes.csv",index=False)